In [4]:
import json
import requests
from typing import Tuple, Dict, List, Union
from collections import Counter

import numpy as np
import networkx as nx
import altair as alt
import seaborn as sns
import pandas as pd
from IPython.display import display

In [9]:
import nltk
nltk.download(['punkt', 'averaged_perceptron_tagger', 'stopwords'])

from nltk.util import ngrams
from textblob import TextBlob

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/hongdonghee/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/hongdonghee/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hongdonghee/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [6]:
# nodes = pd.read_csv('https://raw.githubusercontent.com/jjmmwon/VAST-Challenge-2023-MC3/main/data/all_unique_nodes.csv')
nodes = pd.read_csv('./data/all_unique_nodes.csv')
links = pd.read_csv('./data/links_with_weight.csv')
# links = pd.read_csv('https://raw.githubusercontent.com/jjmmwon/VAST-Challenge-2023-MC3/main/data/links_with_weight.csv')

In [7]:
nodes['product_services'] = nodes['product_services'].apply(eval)

In [11]:
# nodes['product_services_string'] = nodes['product_services'].apply(' | '.join)
# nodes['product_services_string']
nodes['product_services_string'] = nodes['id'].astype(str) 
# + ' | ' + nodes['product_services'].apply(' | '.join)
# nodes

In [12]:
# nodes["sentences"] = nodes.product_services_string.apply(lambda x: nltk.sent_tokenize(x))
# nodes["num_sentences"] = nodes.sentences.apply(lambda x: len(x))
# nodes["sentences"]
nodes['words'] = nodes.product_services_string.apply(lambda x: nltk.word_tokenize(x))
nodes['tagged_words'] = nodes.words.apply(lambda x: nltk.pos_tag(x))

In [13]:
nodes[['words', 'tagged_words']]

,words,tagged_words
0,"[Jones, LLC]","[(Jones, NNP), (LLC, NNP)]"
1,"[Coleman, ,, Hall, and, Lopez]","[(Coleman, NNP), (,, ,), (Hall, NNP), (and, CC..."
2,"[Aqua, Advancements, Sashimi, SE, Express]","[(Aqua, NNP), (Advancements, NNP), (Sashimi, N..."
3,"[Makumba, Ltd, ., Liability, Co]","[(Makumba, NNP), (Ltd, NNP), (., .), (Liabilit..."
4,"[Taylor, ,, Taylor, and, Farrell]","[(Taylor, NN), (,, ,), (Taylor, NNP), (and, CC..."
...,...,...
51726,"[Katherine, Stewart]","[(Katherine, NNP), (Stewart, NNP)]"
51727,"[Christopher, James]","[(Christopher, NNP), (James, NNP)]"
51728,"[Jon, Maddox]","[(Jon, NNP), (Maddox, NNP)]"
51729,"[Sandra, Eaton]","[(Sandra, NNP), (Eaton, NNP)]"


In [18]:
nodes['nouns'] = nodes.tagged_words.apply(lambda x: [word for word, tag in x if tag.startswith("NN")])
nodes[["tagged_words", "nouns"]]

,tagged_words,nouns
0,"[(Jones, NNP), (LLC, NNP)]","[Jones, LLC]"
1,"[(Coleman, NNP), (,, ,), (Hall, NNP), (and, CC...","[Coleman, Hall, Lopez]"
2,"[(Aqua, NNP), (Advancements, NNP), (Sashimi, N...","[Aqua, Advancements, Sashimi, SE, Express]"
3,"[(Makumba, NNP), (Ltd, NNP), (., .), (Liabilit...","[Makumba, Ltd, Liability, Co]"
4,"[(Taylor, NN), (,, ,), (Taylor, NNP), (and, CC...","[Taylor, Taylor, Farrell]"
...,...,...
51726,"[(Katherine, NNP), (Stewart, NNP)]","[Katherine, Stewart]"
51727,"[(Christopher, NNP), (James, NNP)]","[Christopher, James]"
51728,"[(Jon, NNP), (Maddox, NNP)]","[Jon, Maddox]"
51729,"[(Sandra, NNP), (Eaton, NNP)]","[Sandra, Eaton]"


In [19]:
# Your answer here
from collections import Counter

def get_counter(dataframe, stopwords=[], target_tag=None):
    counter = Counter()
    
    for l in dataframe.tagged_words:
        word_set = set()

        for t in l:
            word = t[0].lower()
            tag = t[1]

            ##########################################################
            # Check if the word is a stopword.
            ##########################################################
            if word in stopwords:
                continue

            if target_tag is None:
                word_set.add(word)
            else:
                ##########################################################
                # Check the tag
                ##########################################################
                if tag.startswith(target_tag):
                    word_set.add(word)
                else:
                    continue

        counter.update(word_set)
        
    return counter

global_stopwords = nltk.corpus.stopwords.words("english") 
counter1 = get_counter(nodes, global_stopwords, "NN")
len(counter1.most_common())

10509

In [20]:
import string
local_stopwords = [c for c in string.punctuation] +\
                  ['‘', '’', '—', '…', "'s", "n't", '“', 'unknown']
counter2 = get_counter(nodes, global_stopwords+local_stopwords, "NN")
len(counter2.most_common())

10508

In [17]:
counter3 = get_counter(nodes, global_stopwords+local_stopwords, "JJ")
len(counter3.most_common())

1746

In [18]:
filtered_counter2 = {word: count for word, count in counter2.items() if count > 2}
len(filtered_counter2)

1904

In [19]:
filtered_counter3 = {word: count for word, count in counter3.items() if count > 2}
len(filtered_counter3)

401

In [21]:
import pandas as pd

df2 = pd.DataFrame(list(filtered_counter2.items()), columns=['category', 'count'])
df3 = pd.DataFrame(list(filtered_counter3.items()), columns=['category', 'count'])

In [23]:
df = pd.concat([df2, df3], axis=0, join='outer', ignore_index=False, keys=None)
df.to_csv('words.csv', index=False)

In [1]:
import pandas as pd
from collections import Counter
ocean_words = pd.read_csv('./ocean_words.csv')
ocean_words = ocean_words[ocean_words['label']==1]
ocean_words_list =list(Counter(ocean_words['category']).keys())
ocean_words_list

['water',
 'vessels',
 'liner',
 'sand',
 'pacific',
 'ports',
 'salt',
 'cod',
 'sea',
 'squid',
 'hooks',
 'hook',
 'shipping',
 'fresh',
 'seal',
 'seafood',
 'shrimp',
 'salts',
 'salmon',
 'fish',
 'mediterranean',
 'ice',
 'maritime',
 'hatcheries',
 'liners',
 'fishing',
 'ocean',
 'carriers',
 'shippers',
 'marine',
 'scales',
 'seafoods',
 'fins',
 'shellfish',
 'fillets',
 'pollock',
 'trout',
 'deck',
 'mollusks',
 'crustaceans',
 'lobster',
 'clams',
 'bass',
 'tuna',
 'crabs',
 'tilapia',
 'oysters',
 'fishmeal',
 'atlantic',
 'mackerel',
 'sardines',
 'cruise',
 'ship',
 'shrimps',
 'shell',
 'anchors',
 'herring',
 'halibut',
 'haddock',
 'whiting',
 'saithe',
 'capelin',
 'mussels',
 'urchins',
 'lobsters',
 'fishes',
 'aquaculture',
 'waters',
 'tanks',
 'yellowfin',
 'roughy',
 'snapper',
 'sole',
 'dory',
 'shark',
 'scampi',
 'chum',
 'sushi',
 'scallop',
 'crab',
 'fisheries',
 'whitefish',
 'fillet',
 'squids',
 'prawns',
 'molluscs',
 'shells',
 'cephalopods',
 '

In [2]:
len(ocean_words_list)
ocean_words_list.remove('freeze')
ocean_words_list.remove('habitat')
ocean_words_list.remove('frozen')
ocean_words_list.remove('alaskan')
ocean_words_list.remove('bay')
ocean_words_list.remove('anchors')
ocean_words_list.remove('anchor')
ocean_words_list.remove('cruise')
ocean_words_list.remove('ship')
ocean_words_list.remove('ships')
ocean_words_list.remove('vessels')
ocean_words_list.remove('liner')
ocean_words_list.remove('sand')
ocean_words_list.remove('ports')
ocean_words_list.remove('hook')
ocean_words_list.remove('shipping')
ocean_words_list.remove('carriers')
ocean_words_list.remove('deck')
ocean_words_list.remove('liners')
ocean_words_list.remove('fresh')
ocean_words_list.remove('buoys')
ocean_words_list.remove('water')
ocean_words_list.remove('seal')
ocean_words_list.remove('boats')
ocean_words_list.remove('shippers')
ocean_words_list.remove('salts')
ocean_words_list.remove('net')
ocean_words_list.remove('nets')
ocean_words_list.remove('waters')
ocean_words_list.remove('tanks')

ocean_words_list.remove('ice')
ocean_words_list.remove('smoked')
ocean_words_list.remove('hooks')
ocean_words_list.remove('scales')
ocean_words_list.remove('salt')
len(ocean_words_list)

143

In [9]:
nodes = pd.read_csv('./data/all_unique_nodes.csv')
nodes['product_services'] = nodes['product_services'].apply(eval)

In [19]:
def is_related(row):
    for word in ocean_words_list:
        if any(word.lower() == product.lower() for product in row['words']):
            return 1
    return 0

nodes['related'] = nodes.apply(is_related, axis=1)
nodes

,id,country,product_services,revenue_omu,type,type_length,is_source,is_target,weighted_degree,degree,...,degree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,product_services_string,sentences,num_sentences,words,tagged_words,related
0,Jones LLC,ZH,"[Automobiles, Unknown]","['None', 310612303.447]","{'Company': 1, 'Beneficial Owner': 5, 'Company...",11,1,0,8,8,...,1.000000,1.000000,1.000000,7.071062e-01,Jones LLC | Automobiles | Unknown,[Automobiles | Unknown],1,"[Jones, LLC, |, Automobiles, |, Unknown]","[(Jones, NNP), (LLC, NNP), (|, NNP), (Automobi...",0
1,"Coleman, Hall and Lopez",ZH,"[Passenger cars, trucks, vans, and buses]",[162734683.9969],{'Company': 1},1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000e+00,"Coleman, Hall and Lopez | Passenger cars, truc...","[Passenger cars, trucks, vans, and buses]",1,"[Coleman, ,, Hall, and, Lopez, |, Passenger, c...","[(Coleman, NNP), (,, ,), (Hall, NNP), (and, CC...",0
2,Aqua Advancements Sashimi SE Express,Oceanus,[Holding firm whose subsidiaries are engaged i...,[115004666.6728],{'Company': 1},1,1,0,3,3,...,1.000000,1.000000,1.000000,7.071069e-01,Aqua Advancements Sashimi SE Express | Holding...,[Holding firm whose subsidiaries are engaged i...,1,"[Aqua, Advancements, Sashimi, SE, Express, |, ...","[(Aqua, NNP), (Advancements, NNP), (Sashimi, N...",0
3,Makumba Ltd. Liability Co,Utoporiana,"[Car service, car parts and accessories, autom...",[90986412.5191],{'Company': 1},1,1,0,13,13,...,0.001833,0.005912,0.058608,5.826009e-07,"Makumba Ltd. Liability Co | Car service, car p...","[Car service, car parts and accessories, autom...",1,"[Makumba, Ltd, ., Liability, Co, |, Car, servi...","[(Makumba, NNP), (Ltd, NNP), (., .), (Liabilit...",0
4,"Taylor, Taylor and Farrell",ZH,[Fully electric vehicles (EVs) and electric ve...,[81466666.6728],{'Company': 1},1,0,0,0,0,...,0.000000,0.000000,0.000000,0.000000e+00,"Taylor, Taylor and Farrell | Fully electric ve...",[Fully electric vehicles (EVs) and electric ve...,1,"[Taylor, ,, Taylor, and, Farrell, |, Fully, el...","[(Taylor, NN), (,, ,), (Taylor, NNP), (and, CC...",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51726,Katherine Stewart,Unknown,[Unknown],['Unknown'],{'Beneficial Owner': 1},1,0,1,1,1,...,1.000000,0.000000,1.000000,7.071068e-01,Katherine Stewart | Unknown,[Unknown],1,"[Katherine, Stewart, |, Unknown]","[(Katherine, NNP), (Stewart, NNP), (|, NNP), (...",0
51727,Christopher James,Unknown,[Unknown],['Unknown'],{'Company Contacts': 1},1,0,1,1,1,...,1.000000,0.000000,1.000000,7.071068e-01,Christopher James | Unknown,[Unknown],1,"[Christopher, James, |, Unknown]","[(Christopher, NNP), (James, NNP), (|, NNP), (...",0
51728,Jon Maddox,Unknown,[Unknown],['Unknown'],{'Beneficial Owner': 1},1,0,1,1,1,...,0.000141,0.000000,0.072768,5.606578e-09,Jon Maddox | Unknown,[Unknown],1,"[Jon, Maddox, |, Unknown]","[(Jon, NNP), (Maddox, NNP), (|, NNP), (Unknown...",0
51729,Sandra Eaton,Unknown,[Unknown],['Unknown'],{'Company Contacts': 1},1,0,1,1,1,...,0.000141,0.000000,0.048904,3.747188e-14,Sandra Eaton | Unknown,[Unknown],1,"[Sandra, Eaton, |, Unknown]","[(Sandra, NNP), (Eaton, NNP), (|, NNP), (Unkno...",0


In [ ]:
is_ocean_node = []
for name, product in zip(nodes['id'], nodes['product_services']):
    flag = 0
    for word in ocean_words_list:
        if ' '+word+' ' in ' '+name+' ':
            print("name : ", name)
            print("ocean word : ", word)
            print()
            flag = 1
            break
    if flag==0:
        for p in product:
            if flag==1:
                break
            for word in ocean_words_list:
                if (' '+word+' ' in ' '+p+' ') or (' '+word+' ' in p+' '):
                    print("product : ", p)
                    print("ocean word : ", word)
                    print()
                    flag = 1
                    break
    is_ocean_node.append(flag)
is_ocean_node
nodes['related'] = is_ocean_node

In [20]:
# pd.set_option('display.max_seq_items', None)
ocean_nodes = nodes[nodes['related']==1]
ocean_nodes

,id,country,product_services,revenue_omu,type,type_length,is_source,is_target,weighted_degree,degree,...,degree_centrality,betweenness_centrality,closeness_centrality,eigenvector_centrality,product_services_string,sentences,num_sentences,words,tagged_words,related
6,Punjab s Marine conservation,Riodelsol,"[Beef, pork, chicken, and lamb by-products; co...",[72167572.0026],{'Company': 1},1,1,0,6,6,...,0.666667,0.833333,0.600000,6.858327e-01,"Punjab s Marine conservation | Beef, pork, chi...","[Beef, pork, chicken, and lamb by-products; co...",1,"[Punjab, s, Marine, conservation, |, Beef, ,, ...","[(Punjab, NNP), (s, PRP), (Marine, NNP), (cons...",1
12,Saltwater Sisters AS Marine life,Coralmarica,"[Food and daily household needs products, gene...",[41542564.0136],{'Company': 1},1,1,0,3,3,...,1.000000,1.000000,1.000000,7.071069e-01,Saltwater Sisters AS Marine life | Food and da...,"[Food and daily household needs products, gene...",1,"[Saltwater, Sisters, AS, Marine, life, |, Food...","[(Saltwater, NN), (Sisters, NNPS), (AS, NNP), ...",1
13,The Sea Turtle Company,Oceanus,[Grocery products],[40362666.6728],{'Company': 1},1,1,0,78,39,...,0.464286,0.751004,0.442105,7.043013e-01,The Sea Turtle Company | Grocery products,[Grocery products],1,"[The, Sea, Turtle, Company, |, Grocery, products]","[(The, DT), (Sea, NNP), (Turtle, NNP), (Compan...",1
19,Uttar Pradesh Sea Company Seafoam,Osterivaria,[Wide range of consumer products],[24120481.1023],{'Company': 1},1,1,0,12,3,...,0.750000,0.250000,0.800000,4.999997e-01,Uttar Pradesh Sea Company Seafoam | Wide rang...,[Wide range of consumer products],1,"[Uttar, Pradesh, Sea, Company, Seafoam, |, Wid...","[(Uttar, NNP), (Pradesh, NNP), (Sea, NNP), (Co...",1
22,Okavango River NV Marine life,Kondanovia,[Batteries],[15884149.6701],{'Company': 1},1,1,0,7,7,...,0.000987,0.001691,0.049070,3.610137e-15,Okavango River NV Marine life | Batteries,[Batteries],1,"[Okavango, River, NV, Marine, life, |, Batteries]","[(Okavango, NNP), (River, NNP), (NV, NNP), (Ma...",1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46566,Julie Bass,Unknown,[Unknown],['Unknown'],{'Beneficial Owner': 1},1,0,1,1,1,...,0.000141,0.000000,0.061534,1.325330e-13,Julie Bass | Unknown,[Unknown],1,"[Julie, Bass, |, Unknown]","[(Julie, NNP), (Bass, NNP), (|, NNP), (Unknown...",1
47526,Katherine Herring,Unknown,[Unknown],['Unknown'],{'Beneficial Owner': 1},1,0,1,1,1,...,0.000141,0.000000,0.068013,1.833378e-09,Katherine Herring | Unknown,[Unknown],1,"[Katherine, Herring, |, Unknown]","[(Katherine, NNP), (Herring, NNP), (|, NNP), (...",1
47851,Melanie Bass,Unknown,[Unknown],['Unknown'],{'Beneficial Owner': 1},1,0,1,1,1,...,1.000000,0.000000,1.000000,7.071068e-01,Melanie Bass | Unknown,[Unknown],1,"[Melanie, Bass, |, Unknown]","[(Melanie, NNP), (Bass, NNP), (|, NNP), (Unkno...",1
49594,Karen Bass,Unknown,[Unknown],['Unknown'],{'Beneficial Owner': 1},1,0,1,1,1,...,0.083333,0.000000,0.521739,2.041244e-01,Karen Bass | Unknown,[Unknown],1,"[Karen, Bass, |, Unknown]","[(Karen, NNP), (Bass, NNP), (|, NNP), (Unknown...",1


In [72]:
ocean_nodes.to_csv("./data/ocean_nodes_new.csv", index=False)

In [74]:
ocean_words

,category,count,label
41,water,72,1.0
171,vessels,10,1.0
238,liner,3,1.0
241,sand,10,1.0
248,pacific,19,1.0
...,...,...,...
2257,octopus,6,1.0
2277,net,3,1.0
2285,freshwater,3,1.0
2286,crawfish,3,1.0


In [5]:
from wordcloud import WordCloud 
from collections import Counter

In [6]:
counter = {}
for wd in ocean_words_list:
    counter[wd] = ocean_words[ocean_words['category'] == wd]['count'].sum()

wc = WordCloud(background_color="white", max_words=152, width=800, height=500)
wc.generate_from_frequencies(counter)
wc.to_file(f"./wordcloud2.png")

In [83]:
len(ocean_words_list)

152